# Семинар_5. Поиск оптимальных гиперпараметров модели. Optuna.

Цель семинара: освоить основные способы подбора гиперпараметров ml моделей

План семинара:

* Практика - попробуем gridsearch подход подбора гиперпараметров
* Практика - запустим random search catboost
* Практика - осуществим подбор гиперпараметров библиотекой optuna
* Подведение итогов - проанализируем и обсудим результаты

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn import metrics

import numpy as np
from xgboost import XGBClassifier
import xgboost as xgb
from catboost import CatBoostClassifier

import pandas as pd
pd.set_option('display.max_columns', None)

# 1. Задача gridsearch sklearn (10 минут)

Как вы уже могли убедиться на предидущих семинарах: изменяя гиперпараметры мы изменяем зачения метрик на валидации.

Gridsearch - простой перебор всех возможных комбинаций из параметров.

Если мы настроили один параметр, не меняя остальные - это не значит, что полученное значение параметра является абсолютно идеальным.

Для начала подготовим данные.

In [2]:
train = pd.read_csv('/kaggle/input/seminar1/train_multiclass.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e3/test.csv')
train.head()

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults,target
0,0,584,590,909972,909977,16,8,5,2274,113,140,1358,0,1,50,0.7393,0.4000,0.5000,0.0059,1.0000,1.0000,0.0,1.2041,0.9031,0.6990,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0,3
1,1,808,816,728350,728372,433,20,54,44478,70,111,1687,1,0,80,0.7772,0.2878,0.2581,0.0044,0.2500,1.0000,1.0,2.6365,0.7782,1.7324,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1,6
2,2,39,192,2212076,2212144,11388,705,420,1311391,29,141,1400,0,1,40,0.0557,0.5282,0.9895,0.1077,0.2363,0.3857,0.0,4.0564,2.1790,2.2095,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0,2
3,3,781,789,3353146,3353173,210,16,29,3202,114,134,1387,0,1,40,0.7202,0.3333,0.3333,0.0044,0.3750,0.9310,1.0,2.3222,0.7782,1.4314,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0,2
4,4,1540,1560,618457,618502,521,72,67,48231,82,111,1692,0,1,300,0.1211,0.5347,0.0842,0.0192,0.2105,0.9861,1.0,2.7694,1.4150,1.8808,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1,6


In [3]:
targets = ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']

X = train.drop(columns=targets + ['target', 'id'])
y = train['target']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                  random_state=43, stratify=y)

Сначала просто запустим xgboost

In [5]:
%%time

params = {
    'objective': 'multi:softprob',
    'tree_method': 'hist',
    'seed': 42,
    'num_class': 8,
}

xgb_model = XGBClassifier(**params)

xgb_model.fit(X_train, y_train,
              verbose=5)

y_pred = xgb_model.predict_proba(X_test)
roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovo')
roc_auc

CPU times: user 10.4 s, sys: 85.9 ms, total: 10.5 s
Wall time: 2.73 s


0.8448449170043777

Теперь подберем параметры  с помощью gridsearch
* Импортируйте нужный модуль из sklearn.model_selection
* Задайте пространство возможных параметров (в [документации](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection) посмотрите, как это нужно делать)
* Проследите, чтобы отбор гиперпараметров происходил по подходящей для нас метрике
* Запустите и выведите лучшие параметры и лучший скор
* Для экономии времени не добавляйте слишком много параметров достаточно суммарно 3-4 экспериментов

In [9]:
%%time

from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'objective': ['multi:softprob'],
    'tree_method': ['hist'],
    'seed': [42],
    'num_class': [8],
    'n_estimators': [80, 110],
    'max_depth': [3, 6],
}

xgb_model = XGBClassifier()

grid_search = GridSearchCV(estimator=xgb_model,
                           param_grid=param_grid,
                           scoring='roc_auc_ovo',
                           cv=3)

#For integer/None inputs, if the estimator is a classifier and 
#y is either binary or multiclass, StratifiedKFold is used.

grid_search.fit(X_train, y_train,
                verbose=20)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'max_depth': 3, 'n_estimators': 80, 'num_class': 8, 'objective': 'multi:softprob', 'seed': 42, 'tree_method': 'hist'}
Best Score: 0.8588065761252519
CPU times: user 1min 20s, sys: 486 ms, total: 1min 21s
Wall time: 21.2 s


In [8]:
best_params = grid_search.best_params_

best_xgb_model = XGBClassifier(**best_params)

best_xgb_model.fit(X_train, y_train,
                    verbose=5)

y_pred = best_xgb_model.predict_proba(X_test)

roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovo')
roc_auc

0.8593577604565884

# 2. random search (10 мин)

В отличие от GridSearchCV, не все значения параметров проверяются. Из указанных распределений проверяется фиксированное количество вариантов параметров.

Если все параметры представлены в виде списка, выполняется выборка без замены. Если хотя бы один параметр задан как распределение, используется выборка с заменой. В документации рекомендуется использовать непрерывные распределения для непрерывных параметров.

* Запустите метод randomized_search из катбуста
* Найдите в [документации](https://catboost.ai/en/docs/concepts/python-reference_catboostclassifier_randomized_search) каким параметром управляется кол-во экспериментов
* Посмотрите как можно задавать гиперпараметры. Попробуйте задать как списком, так и распределением
* Отключите логирование, чтобы не засорять ноутбук

In [21]:
from scipy.stats import randint

model = CatBoostClassifier(task_type='GPU',
                          logging_level='Silent',
                           random_seed=42,
                          )

grid = {'learning_rate': [0.03, 0.1],
       # 'depth': [4, 8],
       'depth': randint(3, 10),
}

randomized_search_result = model.randomized_search(grid,
                                                   cv=3,
                                                   X=X_train, 
                                                   y=y_train,
                                                   n_iter=3,
                                                   plot=False)

randomized_search_result['params']

0:	loss: 1.0619740	best: 1.0619740 (0)	total: 4.42s	remaining: 8.85s
1:	loss: 1.0646326	best: 1.0619740 (0)	total: 7.1s	remaining: 3.55s
2:	loss: 1.0574348	best: 1.0574348 (2)	total: 12.9s	remaining: 0us
Estimating final quality...


{'depth': 6.0, 'learning_rate': 0.03}

* Запустим тренировку с подобранными гиперпараметрами

In [14]:
best_params = randomized_search_result['params']
model = CatBoostClassifier(task_type='GPU',random_seed=42, **best_params)

model.fit(X_train, y_train, verbose=100)

y_pred = model.predict_proba(X_test)

roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovo')
roc_auc

0:	learn: 2.0127939	total: 6.74ms	remaining: 6.73s
100:	learn: 1.0779364	total: 575ms	remaining: 5.12s
200:	learn: 1.0068419	total: 1.14s	remaining: 4.54s
300:	learn: 0.9703119	total: 1.72s	remaining: 3.99s
400:	learn: 0.9434083	total: 2.3s	remaining: 3.43s
500:	learn: 0.9189095	total: 2.86s	remaining: 2.85s
600:	learn: 0.8978186	total: 3.44s	remaining: 2.28s
700:	learn: 0.8786987	total: 4s	remaining: 1.71s
800:	learn: 0.8587561	total: 4.63s	remaining: 1.15s
900:	learn: 0.8396941	total: 5.2s	remaining: 572ms
999:	learn: 0.8218556	total: 5.79s	remaining: 0us


0.8606725186239743

Можно также использовать RandomizedSearchCV из sklearn

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

param_distributions = {
    'depth': randint(3, 10),
    'l2_leaf_reg': uniform(0.1, 10),
    'task_type': ['GPU'],
}

catboost_model = CatBoostClassifier()

random_search = RandomizedSearchCV(estimator=catboost_model, 
                                   param_distributions=param_distributions,
                                   scoring='roc_auc_ovo',
                                   cv=2, n_iter=3,
                                   random_state=42)

#random_search.fit(X_train, y_train, verbose = 10)

#best_params = random_search.best_params_
#best_score = random_search.best_score_

# 3. Optuna. Разминка  (10 мин)

* Библиотека с SoTA алгоритмами для подбора гиперпараметров
* Есть ранняя остановка экспериметнтов (pruners)
* Легко распараллеливается
* Можно прикрутить dashboard с визуализацией обучения в реальном времени
* Может быть интегрирован в большинство популярных ml библиотек (со списком можно ознакомиться в [документации](https://optuna.readthedocs.io/en/stable/reference/integration.html#optuna-integration) )

Для начала попробуем минимизировать функцию:
* Нам нужно создать objective, который ссодержит пространство гипперпараметров и возвращает значение, которое нам нужно минимизировать.
* Далее создаем study. И запускаем с нашим objective. Параметр n_trials контролирует кол-во экспериментов.

In [15]:
%%time
import optuna

def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    return (x - 2) ** 2

study = optuna.create_study()
study.optimize(objective, n_trials=10)

study.best_params

[I 2024-03-18 16:07:14,139] A new study created in memory with name: no-name-c11ae39a-b2c2-4002-bbcc-4ba94dc18832
[I 2024-03-18 16:07:14,142] Trial 0 finished with value: 34.57644200582144 and parameters: {'x': -3.880173637387032}. Best is trial 0 with value: 34.57644200582144.
[I 2024-03-18 16:07:14,143] Trial 1 finished with value: 51.26738565728312 and parameters: {'x': 9.160124695651824}. Best is trial 0 with value: 34.57644200582144.
[I 2024-03-18 16:07:14,144] Trial 2 finished with value: 43.707242461660385 and parameters: {'x': -4.61114532147497}. Best is trial 0 with value: 34.57644200582144.
[I 2024-03-18 16:07:14,146] Trial 3 finished with value: 31.451168767020555 and parameters: {'x': 7.608134160932721}. Best is trial 3 with value: 31.451168767020555.
[I 2024-03-18 16:07:14,147] Trial 4 finished with value: 17.647404487804863 and parameters: {'x': -2.2008813941606187}. Best is trial 4 with value: 17.647404487804863.
[I 2024-03-18 16:07:14,148] Trial 5 finished with value: 3

CPU times: user 817 ms, sys: 50.7 ms, total: 867 ms
Wall time: 1.04 s


{'x': 2.183847935300612}

Результат не очень хороший. 
* Давайте попробуем получить результат получше - увеличьте кол-во экспериментов
* Уберите логирование для каждого эксперимета, чтобы не засорять ноутбук (смотрим [доку](https://optuna.readthedocs.io/en/stable/reference/logging.html))


In [16]:
%%time
optuna.logging.set_verbosity(optuna.logging.CRITICAL)
study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params

CPU times: user 572 ms, sys: 1.97 ms, total: 574 ms
Wall time: 573 ms


{'x': 2.0009728775266553}

* Представим, что нам нужно максимизировать функцию.
* Поменяйте код, под задачу максимизации (область определения оставим неизменной (от -10 до 10)

In [20]:
def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    return (x - 2) ** 2

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

study.best_params

{'x': -9.999301869533804}

# 4. Optuna. Углубляемся (20 мин)

Байесовская оптимизация - итерационный метод, который на каждой итерации указывает наиболее вероятную точку, в которой наша целевая функция будет оптимальна. При этом выдаваемые вероятные точки включают две компоненты:

* хорошая точка там, где согласно истории функция выдавала хорошие результаты на предыдущих вызовах (exploitation) 

* хорошая точка там, где высокая неопределенность, то есть неисследованные части пространства (exploration)

Основное отличие от gridsearch и randomsearch в том, что для выбора следующих экспериментов учитываются предыдущие.

Подберите [гиперпараметры](https://lightgbm.readthedocs.io/en/latest/Parameters.html#core-parameters) для XGBClassifier с помощью Optuna.

**Обязательно добавьте по одному: целый, непрерывный и категориальный параметры**

Учитывайте типы параметров:
* suggest_categorical(name, choice) задает категориальные параметры. 
* suggest_float(name, low, high, *, step=None, log=False) задает параметр типа float - число с плавающей точкой.
* suggest_int(name, low, high, step=1, log=False) задает параметр типа int - целое число. 

In [26]:
%%time
def objective(trial):
    params = {
        'objective': 'multi:softprob',
        'tree_method': 'hist',
        'seed': 42,
        'num_class': 8,
        'n_estimators': trial.suggest_categorical('n_estimators', [50, 90]),
        'max_depth': trial.suggest_int('max_depth', 2, 7, step=1),
        'boosting': trial.suggest_categorical('boosting', ['gbdt', 'dart']),
    }

    xgb_model = XGBClassifier(**params)
    xgb_model.fit(X_train, y_train,
                  verbose=20)

    y_pred = xgb_model.predict_proba(X_test)
    score = roc_auc_score(y_test, y_pred, multi_class='ovo')
    return score

study = optuna.create_study(direction= 'maximize')
study.optimize(objective, n_trials=10)


best_params = study.best_params
best_score = study.best_value

print("Best Parameters:", best_params)
print("Best Score:", best_score)

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:30:38] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "boosting" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:30:39] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "boosting" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:30:40] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "boosting" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:30:45] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "boosting" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:30:47] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "boosting" } are not used.

  warnings.warn(smsg

Best Parameters: {'n_estimators': 90, 'max_depth': 2, 'boosting': 'gbdt'}
Best Score: 0.8622484839318804
CPU times: user 1min 3s, sys: 260 ms, total: 1min 3s
Wall time: 16.6 s


In [23]:
best_params = study.best_params

best_xgb_model = XGBClassifier(**best_params)
best_xgb_model.fit(X_train, y_train)

y_pred_proba = best_xgb_model.predict_proba(X_test)

score = roc_auc_score(y_test, y_pred_proba, multi_class='ovo')
score

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:24:38] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "boosting_type" } are not used.

  warnings.warn(smsg, UserWarning)


0.8622484839318804

## Sampler
Посмотрите в [доке](https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/003_efficient_optimization_algorithms.html) какой алгоритм используется оптюной по-умолчанию.
* Попробуйте запустить подбор параметров с другим алгоритмом.
* Подробнее про умную оптимизацию гиперпараметров можно почитать в [учебнике](https://education.yandex.ru/handbook/ml/article/podbor-giperparametrov) от яндекс
* Измерьте время работы алгоритма

In [24]:
%%time
def objective(trial):
    params = {
        'objective': 'multi:softprob',
        'tree_method': 'hist',
        'seed': 42,
        'num_class': 8,
        'n_estimators': trial.suggest_categorical('n_estimators', [50, 90]),
        'max_depth': trial.suggest_int('max_depth', 2, 7, step=1),
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart']),
    }

    xgb_model = XGBClassifier(**params)
    xgb_model.fit(X_train, y_train,
                  verbose=20)

    y_pred = xgb_model.predict_proba(X_test)
    score = roc_auc_score(y_test, y_pred, multi_class='ovo')
    return score

study = optuna.create_study(sampler=optuna.samplers.CmaEsSampler(), 
                            direction='maximize'
                           )

study.optimize(objective, n_trials=15)

best_params = study.best_params
best_score = study.best_value

print("Best Parameters:", best_params)
print("Best Score:", best_score)

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:26:52] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "boosting_type" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:26:53] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "boosting_type" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:26:54] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "boosting_type" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:26:55] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "boosting_type" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:26:56] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "boosting_type" } are not us

Best Parameters: {'n_estimators': 90, 'max_depth': 2, 'boosting_type': 'dart'}
Best Score: 0.8622484839318804
CPU times: user 1min 23s, sys: 350 ms, total: 1min 23s
Wall time: 21.6 s


## Pruner
Pruners в Optuna - это набор алгоритмов для прореживания экспериментов. Pruning - это механизм который позволяет обрывать эксперименты, которые с большой долей вероятности приведут к неоптимальным результатам.

* Добавте прунер в наш сетап
* Сравните время работы с пруннингом и без пруннига

In [25]:
%%time
def objective(trial):
    params = {
        'objective': 'multi:softprob',
        'tree_method': 'hist',
        'seed': 42,
        'num_class': 8,
        'n_estimators': trial.suggest_categorical('n_estimators', [50, 90]),
        'max_depth': trial.suggest_int('max_depth', 2, 7, step=1),
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart']),
    }

    xgb_model = XGBClassifier(**params)
    xgb_model.fit(X_train, y_train,
                  verbose=20)

    y_pred = xgb_model.predict_proba(X_test)
    score = roc_auc_score(y_test, y_pred, multi_class='ovo')
    return score


study = optuna.create_study(sampler=optuna.samplers.CmaEsSampler(),
                            pruner=optuna.pruners.MedianPruner(),
                            direction='maximize')

study.optimize(objective, n_trials=15)

best_params = study.best_params
best_score = study.best_value

print("Best Parameters:", best_params)
print("Best Score:", best_score)

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:28:16] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "boosting_type" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:28:17] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "boosting_type" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:28:18] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "boosting_type" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:28:19] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "boosting_type" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:28:20] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "boosting_type" } are not us

Best Parameters: {'n_estimators': 50, 'max_depth': 2, 'boosting_type': 'gbdt'}
Best Score: 0.8612615380247617
CPU times: user 1min 41s, sys: 399 ms, total: 1min 42s
Wall time: 26.5 s


# 5. Финальное задание. (20 мин)
* Выберите параметры для catboost и py-boost, чтобы поставить ДОЛГИЙ эксперимент дома.
* Оцените сколько примерно длится один эксперимент и поставте большое кол-во экспериментов, но чтобы влезть в лимиты kaggle
* Сделайте инференс, с лучшими параметрами.
* *Отправить сабмишн на ЛБ

### Catboost
[Гиперпараметры](https://catboost.ai/en/docs/references/training-parameters/common)

Например,

* iterations=,
* learning_rate=,
* l2_leaf_reg=,
* depth=,

In [ ]:
def objective(trial):
    params = {
        'iterations' : 1400, # Можно не перебирать, есть Easrly-Stopping
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.01),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 50),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.8),

        "auto_class_weights": trial.suggest_categorical("auto_class_weights", ["SqrtBalanced", "Balanced", "None"]),
        "depth": trial.suggest_int("depth", 3, 9),

        #"boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        #"used_ram_limit": "14gb",
        "eval_metric": "Accuracy", # Тоже стоит заранее определиться
    }


    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 20)

    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    catboost_model = CatBoostClassifier(task_type='GPU',random_seed=42, **params)
    catboost_model.fit(X_train, y_train, verbose=20)

    y_pred = catboost_model.predict_proba(X_test)
    score = roc_auc_score(y_test, y_pred, multi_class='ovo')

    return score

study = optuna.create_study()
study.optimize(objective, n_trials=150)

best_params = study.best_params
best_score = study.best_value

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Сохраните результаты

In [ ]:
catboost_model = CatBoostClassifier(**best_params)
catboost_model.fit(X_train, y_train, verbose=20)

y_pred = catboost_model.predict_proba(X_test)
score = roc_auc_score(y_test, y_pred, multi_class='ovo')

## Py-boost

In [27]:
X = train.drop(columns=targets + ['target', 'id'])
y = train[targets]

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                  random_state=43)

In [ ]:
!pip install py-boost -q

In [31]:
from py_boost import GradientBoosting
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/opt/conda/lib/python3.10/site-packages/cudf/utils/_numba.py:110: UserWarning: Using CUDA toolkit version (12, 3) with CUDA driver version (12, 2) requires minor version compatibility, which is not yet supported for CUDA driver versions 12.0 and above. It is likely that many cuDF operations will not work in this state. Please install CUDA toolkit version (12, 2) to continue using cuDF.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [32]:
# Посмотрим список возможных гиперпараметров
model = GradientBoosting('bce')
model.__dict__

{'models': None,
 'nfeats': None,
 'postprocess_fn': <function py_boost.gpu.base.Ensemble._default_postprocess_fn(x)>,
 'base_score': None,
 '_on_device': False,
 'quantization': 'Quantile',
 'quant_sample': 200000,
 'max_bin': 256,
 'min_data_in_bin': 3,
 'seed': 42,
 'params': {'loss': 'bce',
  'metric': None,
  'ntrees': 100,
  'lr': 0.05,
  'min_gain_to_split': 0,
  'lambda_l2': 1,
  'gd_steps': 1,
  'max_depth': 6,
  'min_data_in_leaf': 10,
  'colsample': 1.0,
  'subsample': 1.0,
  'target_splitter': 'Single',
  'multioutput_sketch': None,
  'use_hess': True,
  'quantization': 'Quantile',
  'quant_sample': 2000000,
  'max_bin': 256,
  'min_data_in_bin': 3,
  'es': 100,
  'seed': 42,
  'verbose': 10,
  'callbacks': None,
  'debug': False}}

In [ ]:
def objective(trial):
    
    params = {'loss': 'bce',
             ###}
    model = GradientBoosting(**params)

    model.fit(np.array(X_train), np.array(y_train))
    
    preds = model.predict(X_test)
    score = roc_auc_score(y_test, preds, multi_class='ovo')
    
    return score

study = optuna.create_study()
study.optimize(objective, n_trials=2)

best_params = study.best_params
best_score = study.best_value

print("Best Parameters:", best_params)
print("Best Score:", best_score)

In [ ]:
model = GradientBoosting(loss='bce', **best_params)
model.fit(np.array(X_train), np.array(y_train))
    
preds = model.predict(X_test)